### GAIA optimiation DBSCAN with MCMC scheme

1. using a simple ABC scheme
2. ...

In [1]:

using PyCall
using Distributions
using Statistics , StatsBase

using Distances
using Random

push!(LOAD_PATH,"/home/stephane/Science/cluster/GAIA/master/src")
using GaiaClustering

import PyPlot


## directory
rootdir = "/home/stephane/Science/cluster/GAIA"
wdir    = "/home/stephane/Science/cluster/GAIA/products"

cd(wdir)


#### mutable MC and model

mutable struct mc
    eps::Array{Float64}
    mne::Array{Int32}
    mcl::Array{Int32}
    qc::Array{Float64}
end

mutable struct model
    eps::Float64
    min_nei::Int
    min_cl::Int
end

┌ Info: Recompiling stale cache file /home/stephane/.julia/compiled/v1.0/GaiaClustering.ji for GaiaClustering [top-level]
└ @ Base loading.jl:1184


In [2]:
### generate eps, min_neightbor and min_cluster
###
### aeps,amin_nghb, amin_cluster are the parameter of the prior density for the DBSCAN parameters

function theta(a1eps, a2eps, amin_nghb, amin_cluster)
    eps = TruncatedNormal(a1eps, a2eps,0.1,1000.)
    minneighbor = Poisson(amin_nghb)
    mincluster = Poisson(amin_cluster)
    
    e = rand(eps)          ; pe = pdf(eps,e)
    n = rand(minneighbor)  ; pn = pdf(minneighbor, n)
    c = rand(mincluster)   ; pc = pdf(mincluster, c)
    
    ptotal = pe*pn*pc
    params = model(e, n, c)
    return(params, ptotal)
end


## iterate with random walk and yield the probability
##
function thetaiter(θi::model , a1eps, a2eps, amin_nghb, amin_cluster)
    let   
        eps         = TruncatedNormal(a1eps, a2eps,0.1,1000.)
        minneighbor = Poisson(amin_nghb)
        mincluster  = Poisson(amin_cluster)
    
        eps_rw = TruncatedNormal(θi.eps,0.5, 0.1, 1000.)
        new_e = 0.
        new_mn = 0
        new_mcl = 0
        
        iternotfound =true
        while iternotfound
            new_e = rand(eps_rw)
        
            ap = rand()
            if ap > 0.5
                new_mn = θi.min_nei + 1
            else
                new_mn = θi.min_nei -1 
            end
            ap = rand()
            if ap > 0.5
                new_mcl = θi.min_cl + 1
            else
                new_mcl = θi.min_cl -1 
            end        
        
            if new_e > 0 && new_mn > 0 && new_mcl > 0
                iternotfound = false
            end
        end
    
        pe = pdf(eps,new_e)
        pn = pdf(minneighbor, new_mn)
        pc = pdf(mincluster, new_mcl)
        ptotal = pe * pn * pc
    
        params = model(new_e, new_mn, new_mcl)
        return(params, ptotal)
    end
end   

thetaiter (generic function with 1 method)

In [3]:
## compute the dbscan clusters, the metric and the Metropolis acceptance

function find_clusters(df::GaiaClustering.Df, dfcart::GaiaClustering.Df , m::model)
    let 
        labels = clusters(df.data , m.eps , 20, m.min_nei, m.min_cl)
        if length(labels) == 0
            return(0)
        end
        
    ### metrics of the clusters
        q2d = metric(dfcart, labels, "spatial2d" , 2.0 , 20.0, 20 )
        qv = metric(dfcart, labels, "velocity" , 3.0 , 30.0, 20 )
        qp, qa = metric(dfcart, labels, "HRD" )
    
        nlab = []
        for ilab in labels
            push!(nlab,length(ilab))
        end
    
    
    #### metric for number of stars in the cluster
        qn = []
        for nl in nlab
            push!(qn,log10(nl))
        end
    
        qc = 0.
        for i in 1:length(nlab)
            k1 = q2d[i][1]
            k2 = qv[i][1]
            k3 = qa[i][1]
            k4 = qn[i]
            qq = (2 * k1 + 3 * k2 + k3 ) / 6.0
            qc = max(qq,qc)
        end
        return(qc)
    end
end

find_clusters (generic function with 1 method)

In [4]:
function getdata(filevot)
    voname = filevot

    data       = read_votable(voname)
    df         = filter_data(data)
    dfcart     = add_cartesian(df)
    blck       = [[1,2,3],[4,5], [6,7,8]]
    wghtblck   = [4.0,5.0,1.0]
    norm       = "identity"

    dfcartnorm , scale8 = normalization_PerBlock(dfcart, blck, wghtblck , norm, false) 
    return(dfcartnorm , dfcart)
end

getdata (generic function with 1 method)

In [5]:
## ABC MCMC (following Weyan et al. 2013)
## first simple scheme for testing
##
function abc_mcmc(df::GaiaClustering.Df, dfcart::GaiaClustering.Df, minimumQ::Float64, burnout::Int)
    let 
        mci = mc(zeros(Float64,0),zeros(Int32,0),zeros(Int32,0) , zeros(Float64,0))
        
        initial = true
        th = []
        mi = model(0.0,0,0)
        micurrent = model(0.0,0,0)
        probi = 0.
        probcurrent = 0.
    
        while initial
            mi, probi = theta(1.5,1.5,10,10)
            qres = find_clusters(df, dfcart, mi)
        
            if qres > minimumQ
                println("## MCMC: init done ...")
                initial = false
                push!(mci.eps, mi.eps)
                push!(mci.mne, mi.min_nei)
                push!(mci.mcl, mi.min_cl)
                push!(mci.qc,  qres)
            end
        end
    
        nchain = 1
        loopAgain = true
        while loopAgain
            micurrent, probcurrent = thetaiter(mi , 1.5,1.5,10,10)
            qres = find_clusters(df, dfcart, micurrent)
        
            if qres > minimumQ
            ### Metropolis-Hasting
                α = probcurrent / probi
                if α > rand()
                    mi = micurrent
                    push!(mci.eps, mi.eps)
                    push!(mci.mne, mi.min_nei)
                    push!(mci.mcl, mi.min_cl)
                    push!(mci.qc , qres)
                    nchain += 1
                    if (nchain%500 == 0) println("### chain:",nchain) end
                    if nchain > burnout loopAgain = false end
                end
            end
        end
        return(mci)
    end
end

abc_mcmc (generic function with 1 method)

In [ ]:
### main loop

macro main(niter = 100)
    Random.seed!(1237)
    println("## Main ...")
    
    ## Get the cluster data..
    dfnorm , dfcart = getdata("test.vot")
    
    ## run the mcmc for the dbscan parameters
    minQ = 3.0
    println("## MCMC : Q minimum = ",minQ)
    println("## MCMC : iterations =", niter)
    mc = abc_mcmc(dfnorm, dfcart, minQ, niter)
    
    println("## MCMC done.")
    
    ## plots ################################
    
    PyPlot.plt[:figure](figsize=(12.0,12.0))

    
    nbins = 50
    PyPlot.plt[:subplot](2, 2, 1  )
    h = PyPlot.plt[:hist](mc.eps,nbins)
    PyPlot.plt[:xlabel]("ϵ")
    
    nbins = 20
    PyPlot.plt[:subplot](2, 2, 2 )
    h = PyPlot.plt[:hist](mc.mne,nbins)
    PyPlot.plt[:xlabel]("min_neighbor")  
    
    nbins = 20
    PyPlot.plt[:subplot](2, 2, 3 )
    h = PyPlot.plt[:hist](mc.mcl,nbins)
    PyPlot.plt[:xlabel]("min_cluster") 
    
    nbins = 50
    PyPlot.plt[:subplot](2, 2, 4 )
    h = PyPlot.plt[:hist](mc.qc,nbins)
    PyPlot.plt[:xlabel]("Qc") 
    PyPlot.plt[:show]()
    
    ## Stats
    println("## Stats:")
    println("ϵ : ",mean(mc.eps))
    println("min_nei  : ",mean(mc.mne))
    println("min_clus  : ",mean(mc.mcl))
    println("Qc  : ",mean(mc.qc))   
    
end

@main(10000)

## Main ...
## Votable test.vot read
## Filtering done ...
## Stars selected: 12531
## Cartesian transformation done ...
## Normalization identity done...
### [1pc,1pc,1pc,1km/s,1km/s,1mag,1mag] equivalent to [0.398015, 0.398015, 0.398015, 0.497519, 0.497519, 0.0995037, 0.0995037, 0.0995037]
## MCMC: init done ...
### chain:500
### chain:1000
### chain:1500
### chain:2000
### chain:2500
### chain:3000
### chain:3500
### chain:4000
### chain:4500
### chain:5000
### chain:5500
### chain:6000
### chain:6500
### chain:7000
